In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# 적절한 하이퍼파라미터를 탐색하기 위해 train/valie/test set로 나누기 (보통 6:2:2)

In [35]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [36]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [39]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [13]:
from sklearn.model_selection import train_test_split

In [43]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [45]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [47]:
sub_input.shape

(4157, 3)

In [49]:
val_input.shape

(1040, 3)

In [51]:
test_input.shape

(1300, 3)

In [53]:
from sklearn.tree import DecisionTreeClassifier

In [55]:
dt = DecisionTreeClassifier(random_state = 42)

In [57]:
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [59]:
dt.score(sub_input, sub_target)

0.9971133028626413

In [61]:
dt.score(val_input, val_target)

0.864423076923077

#### 교차 검증

In [64]:
from sklearn.model_selection import cross_validate

In [66]:
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.01147699, 0.00838923, 0.01130867, 0.00872159, 0.00949669]),
 'score_time': array([0.00199413, 0.00392318, 0.00099754, 0.00299668, 0.00198817]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [68]:
# fit_time = 모델을 훈련하는 시간
# score_time = 모델을 검증하는 시간
# test_score = 각각의 테스트 점수

In [72]:
np.mean(scores['test_score'])

0.855300214703487

In [78]:
# 교차 검증을 진행하기 위해서는 훈련 세트를 섞기 위해 분할기를 지정 (교차 검증에서 폴드를 어떻게 나눌지)
#분류 모델일경우 StratifiedKFold
# 회귀 모델일 경우 KFold
from sklearn.model_selection import StratifiedKFold

In [80]:
# 훈련세트를 섞은 후 10-폴드 교차 검증
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8574181117533719

#### 하이퍼파라미터 튜닝

In [83]:
from sklearn.model_selection import GridSearchCV

In [109]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # 불순도 감소 최소량

In [115]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
# n_jobs = 1: 사용 가능한 모든 CPU 코어 사용, 병렬 처리를 최대한 활용하여 실행 속도 높임
# n_jobs = 2: 병렬처리를 하지 않고, 한번에 하나의 작업만 실행 (기본 값)
# n_jobs = (양의 정수): 지정된 CPU 코어 수만 사용

In [91]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [95]:
dt = gs.best_estimator_

In [97]:
dt.score(train_input, train_target)

0.9615162593804117

In [99]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [105]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [107]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

In [123]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # 불순도 감소 최소량
          'max_depth': range(5, 20, 1), # 트리의 깊이
          'min_samples_split': range(2, 100, 10)} # 노드를 나누기 위한 최소 샘플 수

In [125]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

In [127]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [131]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [133]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

In [142]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [152]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
         'max_depth': randint(20, 50),
         'min_samples_split': randint(2, 25),
         'min_samples_leaf': randint(1, 25),}

In [154]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter = 100, n_jobs = -1, random_state = 42)

In [156]:
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000242A6F61AC0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000242A6F60F50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000242A881ABA0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000242A6B53FE0>},
                   random_state=42)

In [158]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [164]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [166]:
dt = gs.best_estimator_

In [168]:
dt.score(test_input, test_target)

0.86